In [1]:
from gensim.models import Word2Vec 
model = Word2Vec.load_word2vec_format('word2vec.bin', binary=True)

In [3]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib
import matplotlib.pyplot as plt
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import pairwise_distances
from nltk.util import ngrams as nltk_ngrams
from collections import Counter
import networkx as nx
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

custom_stopwords = ['rt', '&amp', '#', '', '&amp;', '-', 'amp', '.', 'QQQQQQQQQ', 'll','re','ve']
stopwords_english = stopwords.words('english')
stopwords_spanish = stopwords.words('spanish')
tknzr = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

In [4]:
def load_test_data():

    # Test Set
    start_time = start_exec('Loading test data')
    df_test = pd.read_csv('test_data.csv', index_col=False)
    X_test = df_test['Text'].apply(preprocess_text)
    Y_test = df_test['Sentiment']
    stop_exec(start_time)

    return X_test, Y_test

def preprocess_text(text):
    global st
    text = str(text)
    new_text = text.lower()
    new_text = re.sub(r'https://\S+', 'URL_URL_URL', new_text)
    new_text = re.sub(r'http://\S+', 'URL_URL_URL', new_text)
    new_text = re.sub(r'@\S+', 'USER_MENTION_USER', new_text)
    new_text = new_text.replace('rt','')
    new_text = new_text.replace('&amp','')
    new_text = new_text.replace('&amp','')
    new_text = new_text.replace('amp','')
    new_text = re.sub('[^a-zA-Z\s]', '', new_text)
    new_text = re.sub(r'\b\w{1}\b', '', new_text)
    new_text = re.sub(r'(.)\1+', r'\1\1', new_text)
    new_text = new_text.strip()

    #words = [st.stem(w) for w in new_text.split() if w not in custom_stopwords and w not in stopwords_english]
    #words = [st.stem(w) for w in new_text.split()]

    return new_text
    return ' '.join(words)

def map_sentiment_string(s):
    if s == 'positive': return 1
    if s == 'negative': return -1
    return 0

def map_sentiment_float(f):
    if f > 0.3: return 1
    if f < -0.3: return -1
    return 0

def map_sentiment_int(i):
    i = int(i)
    if i == 4: return 1
    if i == 0: return -1
    return 0

def get_tweet_vector(text):
    words = text.split()
    N = len(words)
    vector = np.zeros(300)
    
    if N > 0:
        for word in words:
            if word in model:
                vector += model[word]
        vector /= N
    return vector


In [7]:
df_train = pd.read_csv('train_data.csv')
df_test = pd.read_csv('test_all.csv')
df_test = df_test[df_test[' Sentiment'] != 'neutral']

In [8]:
df_train['Sentiment_raw'] = df_train['Sentiment']
df_train['Sentiment'] = df_train['Sentiment'].apply(map_sentiment_int)
df_test['Sentiment'] = df_test[' Sentiment'].apply(map_sentiment_string)

In [9]:
X_train = df_train['Text']
Y_train = df_train['Sentiment']
X_test = df_test['Text']
Y_test = df_test['Sentiment']

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [8]:
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                      ('tfidf', TfidfTransformer()),
                      ('clf', LR()),
 ])

In [9]:
text_clf = text_clf.fit(X_train, Y_train)

In [18]:
X_test = X_test.apply(preprocess_text)


In [19]:
predicted = text_clf.predict(X_test)

In [22]:
precision_score(Y_test, predicted)

0.66253101736972708

In [11]:
vectors = []
for i in range(len(df_train)):
    tweet_vector = get_tweet_vector(df_train['Text'][i])
    vectors.append(tweet_vector)
vectors = np.array(vectors)    

In [33]:

word2vec_clf = LR().fit(vectors, Y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [39]:
vectors = None
vectors_array = None
gc.collect()
model['good']

array([ 0.04052734,  0.0625    , -0.01745605,  0.07861328,  0.03271484,
       -0.01263428,  0.00964355,  0.12353516, -0.02148438,  0.15234375,
       -0.05834961, -0.10644531,  0.02124023,  0.13574219, -0.13183594,
        0.17675781,  0.27148438,  0.13769531, -0.17382812, -0.14160156,
       -0.03076172,  0.19628906, -0.03295898,  0.125     ,  0.25390625,
        0.12695312, -0.15234375,  0.03198242,  0.01135254, -0.01361084,
       -0.12890625,  0.01019287,  0.23925781, -0.08447266,  0.140625  ,
        0.13085938, -0.04516602,  0.06494141,  0.02539062,  0.05615234,
        0.24609375, -0.20507812,  0.23632812, -0.00860596, -0.02294922,
        0.05078125,  0.10644531, -0.03564453,  0.08740234, -0.05712891,
        0.08496094,  0.23535156, -0.10107422, -0.03564453, -0.04736328,
        0.04736328, -0.14550781, -0.10986328,  0.14746094, -0.23242188,
       -0.07275391,  0.19628906, -0.37890625, -0.07226562,  0.04833984,
        0.11914062,  0.06103516, -0.12109375, -0.27929688,  0.05

In [ ]:
model.most_similar('good')

In [ ]:
2